In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling
%matplotlib inline

In [2]:
#Pandas profiling creates univariant analysis document
pandas_profiling.ProfileReport(pd.read_csv('Telecom_user dataset for interviews.csv'))

In [3]:
predict_churn_set = pd.read_csv('Telecom_user dataset for interviews.csv')

In [4]:
predict_churn_set.shape

(5986, 20)

In [5]:
#Replace all the spaces with null values
predict_churn_set["TotalCharges"] = predict_churn_set["TotalCharges"].replace(" ",np.nan)

#Drop null values in the floating value column "TotalCharges"
predict_churn_set =  predict_churn_set[predict_churn_set["TotalCharges"].notnull()]
predict_churn_set =  predict_churn_set.reset_index()[predict_churn_set.columns]

#Convert this column values to numeric
predict_churn_set["TotalCharges"] = predict_churn_set["TotalCharges"].astype(float)

In [6]:
#check for values
predict_churn_set["Churn"].value_counts().values

array([4389, 1587], dtype=int64)

In [7]:
#one hot encoding is applied to categorical data
predict_churn_set = pd.get_dummies(predict_churn_set,columns =  ['InternetService','PaymentMethod','Contract'],drop_first=True)


In [8]:
#Perform Feature Scaling
from sklearn.preprocessing import StandardScaler

#Perform Feature Scaling on 'tenure', 'MontlyCharges', 'TotalCharges'
#in order to bring them on same scale

standardScaler = StandardScaler()
columns_for_ftscaling=['tenure','MonthlyCharges','TotalCharges']

#Apply the feature scaling operation on dataset using fit_transform() method
predict_churn_set[columns_for_ftscaling] =standardScaler.fit_transform(predict_churn_set[columns_for_ftscaling])

In [9]:
#See subset of values
predict_churn_set.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,Churn,InternetService_DSL,InternetService_Fiber optic,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year
0,1,0,1,1,1.611381,1,1,0,0,0,...,-1.353483,-0.247769,0,0,0,1,0,0,0,1
1,0,0,0,0,0.468468,1,0,0,1,1,...,0.774066,0.736669,0,0,1,1,0,0,0,0
2,0,1,1,0,0.223558,1,1,0,0,0,...,0.335602,0.251453,1,0,1,0,0,0,0,0
3,1,0,0,0,-1.164265,1,0,0,0,0,...,-0.297182,-0.905725,0,1,0,0,1,0,0,0
4,1,0,0,0,-1.245901,1,0,1,0,1,...,-0.378564,-0.958057,0,1,0,0,1,0,0,0


In [10]:

#Create Feature variable X and Target variable y
y = predict_churn_set['Churn']
x = predict_churn_set.drop('Churn',axis='columns')

#data set is made so that 20% is test data
#and 80% is the test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(x,y,test_size=0.2,random_state =50)

In [11]:
#Machine Learning classification model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [12]:
#Fit the Logistic Regression Model
LogisticRegression = LogisticRegression(random_state=59)
LogisticRegression.fit(X_train,y_train)

#predict the value for new, unseen data
pred = LogisticRegression.predict(X_test)

#Find Accuracy using accuracy score method
LogisticRegression_accuracy = round(metrics.accuracy_score(y_test,pred)*100,2)
print(LogisticRegression_accuracy)

80.18


In [13]:
#Fit the Support Vector Machine Model
smodel = SVC(kernel='linear',random_state=50,probability=True)
smodel.fit(X_train,y_train)

#Predict the value for new,unseen data
s_pred = smodel.predict(X_test)

#Find Accuracy using accuracy sscore method
s_accuracy = round(metrics.accuracy_score(y_test,s_pred)*100,2)
print(s_accuracy)

78.51


In [14]:
#Fit the Decision Tree Classification Model
from sklearn.tree import DecisionTreeClassifier
dtmodel = DecisionTreeClassifier(criterion = "gini", random_state =50)
dtmodel.fit(X_train,y_train)

#Predict the value for new, unseen data
dt_pred = dtmodel.predict(X_test)

#Find Accuracy using accuraccy score method
dt_accuracy = round(metrics.accuracy_score(y_test,dt_pred)*100,2)
print(dt_accuracy)

72.83


In [15]:
#Fit the Random Forest Classification Model
from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier(n_estimators = 100, criterion = 'entropy',random_state=0)
rfmodel.fit(X_train,y_train)

#Predict the value for new, unseen data
rf_pred = rfmodel.predict(X_test)

#Find Accuracy using accuracy score method
rf_accuracy = round(metrics.accuracy_score(y_test,rf_pred)*100,2)
print(rf_accuracy)

79.52


In [16]:
#Compare Several models according to their Accuracies
Model_Comparison = pd.DataFrame({
'Model':['Logistic Regression','Support Vector Machine','Decision Tree','Random Forest'],
'Score':[LogisticRegression_accuracy,s_accuracy,dt_accuracy,rf_accuracy]
})



In [17]:
Model_Comparison_df = Model_Comparison.sort_values(by='Score',ascending=False)
Model_Comparison_df = Model_Comparison_df.set_index('Score')
Model_Comparison_df.reset_index()

,Score,Model
0,80.18,Logistic Regression
1,79.52,Random Forest
2,78.51,Support Vector Machine
3,72.83,Decision Tree
